# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
names(boston)

14-element Array{Symbol,1}:
 :Crim   
 :Zn     
 :Indus  
 :Chas   
 :NOx    
 :Rm     
 :Age    
 :Dis    
 :Rad    
 :Tax    
 :PTRatio
 :Black  
 :LStat  
 :MedV   

In [4]:
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


## 切分訓練資料及測試資料

In [5]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [6]:
train = boston[train_ind, :]
test  = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,3.83684,0.0,18.1,0,0.77,6.251,91.1,2.2955,24,666
2,4.34879,0.0,18.1,0,0.58,6.167,84.0,3.0334,24,666
3,3.56868,0.0,18.1,0,0.58,6.437,75.0,2.8965,24,666
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
5,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273
6,0.10574,0.0,27.74,0,0.609,5.983,98.8,1.8681,4,711
7,0.32264,0.0,21.89,0,0.624,5.942,93.5,1.9669,4,437
8,5.58107,0.0,18.1,0,0.713,6.436,87.9,2.3158,24,666
9,0.07244,60.0,1.69,0,0.411,5.884,18.5,10.7103,4,411


In [7]:
model = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error    t value  Pr(>|t|)    Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)   27.7517      5.46749      5.07578     <1e-6    17.0024     38.5011    
Crim          -0.0958467   0.032551    -2.94451     0.0034   -0.159844   -0.0318498 
Zn             0.0413372   0.014642     2.82319     0.0050    0.0125503   0.0701241 
Indus         -0.0142893   0.0633038   -0.225725    0.8215   -0.138748    0.110169  
Chas           3.11534     1.01368      3.07331     0.0023    1.1224      5.10828   
NOx          -13.9683      4.06022    

In [8]:
intercept, slope = coef(model)

14-element Array{Float64,1}:
  27.75173394425151    
  -0.09584672267143146 
   0.04133721370884331 
  -0.014289258194465932
   3.115339269431253   
 -13.968286129329636   
   4.621618177933468   
  -0.01430159380651865 
  -1.451581864297843   
   0.22773194136784042 
  -0.011004374224565835
  -0.8416323562209069  
   0.00919919464763773 
  -0.4426487855073349  

## 預測

In [9]:
predict(model, test)

101-element Array{Union{Missing, Float64},1}:
 18.704714477375852
 19.44772354657918 
 21.91960634184846 
 28.565811267397205
 28.068150679829603
 14.125014448154326
 16.219388552401867
 17.002418082888532
 16.98919765437718 
 22.984721766483506
 28.624313686271197
 26.08546082220487 
 32.17021757816417 
  ⋮                
 21.098714838460104
 18.58613693952338 
 24.751354819028045
 36.3164666327769  
 15.83641523222433 
 19.433197966039565
 20.961807915833624
 25.84307552578898 
 24.625873395847062
 31.520118146449853
 28.35430191935386 
 18.285655424987908

## 模型評估

In [10]:
GLM.r2(model)

0.7636945125316135

In [11]:
GLM.adjr2(model)

0.7558378083446851